# Utilisation du registre MLFlow

En utilisant l'API MLFlow, effectuez les tâches suivantes sur le registre de modèles.

Au préalable, il faut définir les informations de connexion comme on a pu le faire dans `creds.env`.

Remplacez ci-dessous les valeurs `username` et `password`.

In [ ]:
%set_env MLFLOW_TRACKING_URI=https://dagshub.com/m09/landscape-classifier.mlflow
%set_env MLFLOW_TRACKING_USERNAME=username
%set_env MLFLOW_TRACKING_PASSWORD=password

## Création d'un client MLFlow

Créez un client MLFlow et affichez l'adresse de tracking pour vérifier qu'elle est bien récupérée depuis la variable d'environnement définie en haut du notebook.

In [ ]:
# Votre code ici

### Solution

In [1]:
import mlflow

client = mlflow.client.MlflowClient()
client.tracking_uri

'file:///home/mog/repos/shuuchuu/landscape-classifier/mlruns'

## Recherche de modèle

Affichez tous les modèles enregistrés et pour chacun des modèles la version créée la plus récemment.

In [ ]:
# Votre code ici

### Solution

In [ ]:
def latest_version(latest_versions):
  return sorted(latest_versions, key=lambda v: -v.creation_timestamp)[0].version


for model in client.search_registered_models():
  print(model.name, latest_version(model.latest_versions))

## Récupération d'un modèle depuis le registre de modèle

Maintenant que vous savez récupérer depuis Python le nom et la version des modèles présents sur le registre, récupérez en un et vérifiez que c'est bien l'objet que vous anticipez.

Il y a plusieurs manières de charger un modèle si celui-ci dispose de plusieurs *flavors*. Qu'est-ce que cela change ?

In [ ]:
# Votre code ici

### Solution

In [ ]:
import mlflow.keras


def uri_from_name_and_version(name: str, version: str | int) -> str:
  return f"models:/{name}/{version}"


uri = uri_from_name_and_version("lenet-landscape-classifier", 1)


model_keras = mlflow.keras.load_model(model_uri=uri)
model_keras.summary()
model = mlflow.pyfunc.load_model(model_uri=uri)
model

## Ajout d'alias et d'étiquettes

Les alias et les étiquettes (tags) permettent d'organiser les modèles dans le registre de modèles MLFLow. Les alias ciblent forcément une version en particulier alors que les étiquettes peuvent aussi cibler toutes les versions d'un modèle.

Ajoutez un alias `champion` à la dernière version de votre modèle, un tag `domain` à la valeur `cv` pour votre modèle et un tag `size` à la valeur `small` pour la dernière version de votre modèle.

In [ ]:
# Votre code ici

Récupérez maintenant votre modèle en utilisant l'alias plutôt que la version exacte.

### Solution

In [ ]:
client.set_registered_model_alias("lenet-landscape-classifier", "champion", "1")
client.set_registered_model_tag("lenet-landscape-classifier", "domain", "cv")
client.set_model_version_tag("lenet-landscape-classifier", "1", "size", "small")

import mlflow.keras


def uri_from_name_and_alias(name: str, alias: str | int) -> str:
  return f"models:/{name}@{alias}"


uri = uri_from_name_and_alias("lenet-landscape-classifier", "champion")


model_keras = mlflow.keras.load_model(model_uri=uri)
model_keras.summary()

## Changement de phase d'un modèle

Quand vous souhaitez promouvoir un modèle dans votre chaine de mise en production, la méthode recommandée actuellement est de copier le modèle vers un nouveau nom de modèle.

Copiez un modèle de votre choix vers un nouveau nom, qui sera l'ancien préfixé de `staging.`.

In [ ]:
# Votre code ici

### Solution

In [ ]:
client.copy_model_version(
    src_model_uri="models:/lenet-landscape-classifier@champion",
    dst_name="staging.lenet-landscape-classifier",
)